In [1]:
import numpy as np
import pyarrow
import pyarrow.parquet as pq
from pathlib import Path
from functools import partial
import tqdm
import shutil
import pandas as pd
import dask
import dask.array as da
import dask.dataframe as dd
import math
pd.set_option('max_colwidth', 0)
classes = ['retweet', 'reply', 'like', 'retweet_with_comment']
label_cols = ['has_' + c for c in classes]
pred_cols = ['pred_' + c for c in classes]
data_dir = Path('~/recsys2020')

# Evaluation Process

Similar to the challenge: we evaluate the predictions on a few items that build the candidate set. We compare the predictions to the ground truth (which is the initial dataset) and compute some metrics.

We expect the predictions in the following format

tweet_id,user_id,retweet,reply,like,retweet_with_comment

In [2]:
# Use RCE from RecSys challenge

from sklearn.metrics import log_loss, precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
  prec, recall, thresh = precision_recall_curve(gt, pred)
  prauc = auc(recall, prec)
  return prauc

def calculate_ctr(gt):
  positive = len([x for x in gt if x == 1])
  ctr = positive/float(len(gt))
  return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0


In [3]:
# input_file_gt = data_dir / "xgb_v1_preds.parquet/"
input_file_gt = data_dir / "knn_preds.parquet/"
df_gt = dd.read_parquet(str(input_file_gt))

In [4]:
for pred_type in classes:
  gt_arr = df_gt["has_" + pred_type].values.compute()
  pred_arr = df_gt["pred_" + pred_type].values.compute()
  rce = compute_rce(pred_arr, gt_arr)
  prauc = compute_prauc(pred_arr, gt_arr)
  
  print(f"PRAUC {pred_type:20s} = {prauc:8.4f}")
  print(f"RCE   {pred_type:20s} = {rce:6.2f}")

PRAUC retweet              =   0.2249
RCE   retweet              = -363.61
PRAUC reply                =   0.0398
RCE   reply                = -574.34
PRAUC like                 =   0.6446
RCE   like                 = -258.42
PRAUC retweet_with_comment =   0.0120
RCE   retweet_with_comment = -488.51
